In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

def scrape_books_from_page(url, exchange_rate=100):
    """Scrape book data from a single page URL and convert price to INR."""
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve {url}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('article', class_='product_pod')

    page_data = []
    for book in books:
        title = book.h3.a['title']

        # Get price string and extract numeric part
        price_str = book.find('p', class_='price_color').get_text()  # e.g. "£51.77"
        price_num = float(re.sub(r'[^\d.]', '', price_str))  # Extract number

        # Convert to INR and format with ₹ symbol
        price_inr = price_num * exchange_rate
        price = f"₹{price_inr:.2f}"

        rating = book.p['class'][1]  # e.g. 'One', 'Two', ...
        page_data.append([title, price, rating])
    return page_data

def scrape_multiple_pages(base_url, pages=3, exchange_rate=100):
    """Scrape multiple pages of the site with price conversion."""
    all_books = []
    for page_num in range(1, pages + 1):
        if page_num == 1:
            url = base_url
        else:
            url = f"{base_url}catalogue/page-{page_num}.html"

        print(f"Scraping page {page_num} ...")
        books = scrape_books_from_page(url, exchange_rate)
        all_books.extend(books)

        time.sleep(1)  # polite delay between requests

    return all_books

# Base URL of the site
base_url = "http://books.toscrape.com/"

# Set your exchange rate here (GBP to INR)
exchange_rate = 113.56

# Scrape first 5 pages with currency conversion
books_data = scrape_multiple_pages(base_url, pages=5, exchange_rate=exchange_rate)

# Save data to CSV in Colab environment
csv_filename = "/content/books_data.csv"
with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Price (INR)', 'Rating'])
    writer.writerows(books_data)

print(f"\n✅ Scraped {len(books_data)} books with price converted to INR!")
print(f"CSV file saved at: {csv_filename}")


Scraping page 1 ...
Scraping page 2 ...
Scraping page 3 ...
Scraping page 4 ...
Scraping page 5 ...

✅ Scraped 100 books with price converted to INR!
CSV file saved at: /content/books_data.csv
